In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Import the requests library.
import requests
# Import the API key.
from config import weather_api_key
from config import g_key
import sys
from datetime import datetime
import traceback
import gmaps

In [2]:
def yesno_str_to_bool(user_str):
    return (user_str == "yes")

In [3]:
weather_df = pd.read_csv("analysis/WeatherPy_challenge.csv") 

In [4]:
min_temp=float(input("What min temperature are you interested in?"))
max_temp=float(input("What max temperature are you interested in?"))
rain=input("Do you want it to be raining? (yes/no) ")
snow=input("Do you want it to be snowing? (yes/no) ")

What min temperature are you interested in?50
What max temperature are you interested in?80
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [5]:
include_rain = yesno_str_to_bool(rain)
include_snow = yesno_str_to_bool(snow)
valid_temp_range=weather_df[(weather_df["Max Temp"] >= min_temp) & (weather_df["Max Temp"] <= max_temp)]

In [6]:
valid_temp_range.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,walvis bay,na,"04/18/2020, 18:58:15",-22.96,14.51,55.99,93,2,2.24,clear sky,0.0,0.0
3,port elizabeth,za,"04/18/2020, 18:55:17",-33.92,25.57,59.00,100,0,2.24,clear sky,0.0,0.0
6,ponta do sol,cv,"04/18/2020, 18:58:16",32.67,-17.10,57.00,63,4,5.99,clear sky,0.0,0.0
7,quchan,ir,"04/18/2020, 18:58:16",37.11,58.51,52.68,38,0,3.42,clear sky,0.0,0.0
8,salta,ar,"04/18/2020, 18:58:16",-24.79,-65.41,59.00,87,0,4.70,clear sky,0.0,0.0


In [7]:
if include_rain:
    valid_data_rain = valid_temp_range[valid_temp_range["Rain inches (last 3 hrs)"]>0]
else:
    valid_data_rain = valid_temp_range[valid_temp_range["Rain inches (last 3 hrs)"]==0]

In [8]:
valid_data_rain.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,walvis bay,na,"04/18/2020, 18:58:15",-22.96,14.51,55.99,93,2,2.24,clear sky,0.0,0.0
3,port elizabeth,za,"04/18/2020, 18:55:17",-33.92,25.57,59.00,100,0,2.24,clear sky,0.0,0.0
6,ponta do sol,cv,"04/18/2020, 18:58:16",32.67,-17.10,57.00,63,4,5.99,clear sky,0.0,0.0
7,quchan,ir,"04/18/2020, 18:58:16",37.11,58.51,52.68,38,0,3.42,clear sky,0.0,0.0
8,salta,ar,"04/18/2020, 18:58:16",-24.79,-65.41,59.00,87,0,4.70,clear sky,0.0,0.0


In [9]:
if include_snow:
    valid_data = valid_data_rain[valid_data_rain["Snow inches (last 3 hrs)"]>0]
else:
    valid_data = valid_data_rain[valid_data_rain["Snow inches (last 3 hrs)"]==0]

In [10]:
valid_data.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,walvis bay,na,"04/18/2020, 18:58:15",-22.96,14.51,55.99,93,2,2.24,clear sky,0.0,0.0
3,port elizabeth,za,"04/18/2020, 18:55:17",-33.92,25.57,59.00,100,0,2.24,clear sky,0.0,0.0
6,ponta do sol,cv,"04/18/2020, 18:58:16",32.67,-17.10,57.00,63,4,5.99,clear sky,0.0,0.0
7,quchan,ir,"04/18/2020, 18:58:16",37.11,58.51,52.68,38,0,3.42,clear sky,0.0,0.0
8,salta,ar,"04/18/2020, 18:58:16",-24.79,-65.41,59.00,87,0,4.70,clear sky,0.0,0.0


In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = valid_data[["City", "Country", "Max Temp", "Current Description","Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,walvis bay,na,55.99,clear sky,-22.96,14.51,
3,port elizabeth,za,59.00,clear sky,-33.92,25.57,
6,ponta do sol,cv,57.00,clear sky,32.67,-17.10,
7,quchan,ir,52.68,clear sky,37.11,58.51,
8,salta,ar,59.00,clear sky,-24.79,-65.41,
11,rikitea,pf,77.45,broken clouds,-23.12,-134.97,
14,zhezkazgan,kz,57.92,scattered clouds,47.80,67.71,
16,portland,au,54.00,light rain,45.52,-122.68,
17,rikitea,pf,77.45,broken clouds,-23.12,-134.97,
18,kushima,jp,64.40,broken clouds,31.46,131.23,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [13]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    lat_long = f"{lat},{lng}"
    params["location"] = lat_long
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    if len(hotels["results"]) == 0:
        print(f"Hotel results == 0... skipping. {lat_long}")
        continue
    try:
        hotel_result = hotels["results"]
        hotel_df.loc[index, "Hotel Name"] = hotel_result[0]["name"]
    except (IndexError):
        print(f"Hotel not found... skipping. {hotel_result}")
    except:
        print(f"Unexpected error: {sys.exc_info()[0]}")
        print(f"Unexpected error[1]: {sys.exc_info()[1]}")

Hotel results == 0... skipping. 37.83,12.67
Hotel results == 0... skipping. 40.28,97.2
Hotel results == 0... skipping. 11.03,20.28
Hotel results == 0... skipping. 22.68,-3.98
Hotel results == 0... skipping. 33.19,-82.53
Hotel results == 0... skipping. -5.18,-68.9
Hotel results == 0... skipping. 22.68,-3.98
Hotel results == 0... skipping. -25.17,45.05
Hotel results == 0... skipping. -26.4,-52.47
Hotel results == 0... skipping. 26.1,107.47
Hotel results == 0... skipping. 41.46,45.09
Hotel results == 0... skipping. 51.36,80.19
Hotel results == 0... skipping. 4.62,21.89
Hotel results == 0... skipping. -21.84,-51.61
Hotel results == 0... skipping. 8.08,22.4
Hotel results == 0... skipping. 45.77,47.13
Hotel results == 0... skipping. 37.42,66.04
Hotel results == 0... skipping. 8.2,-59.78


In [14]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Descritpion</dt><dd>Description: {Current Description} with a max temp: {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [15]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]

In [16]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#Ensure there is at least 1 location and apply the info boxes to each pin.
if len(locations) > 0:
    # Get the maximum temperature.
    max_temp = hotel_df["Max Temp"]
    #GMaps weights doesn't allow negative values.
    temps = []
    for temp in max_temp:
        temps.append(max(temp, 0))

    #heat_layer = gmaps.heatmap_layer(locations,weights=temps,dissipating=False,max_intensity=300, point_radius=4)
    heat_layer = gmaps.heatmap_layer(locations,weights=temps)
    marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
    fig.add_layer(heat_layer)
    fig.add_layer(marker_layer)
    #Save hotel dataframe to a .CSV
    hotel_df.to_csv('./analysis/WeatherPy_vacation.csv')

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
hotel_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,walvis bay,na,55.99,clear sky,-22.96,14.51,Oyster Box Guesthouse
3,port elizabeth,za,59.00,clear sky,-33.92,25.57,39 On Nile Guest House
6,ponta do sol,cv,57.00,clear sky,32.67,-17.10,Hotel do Campo
7,quchan,ir,52.68,clear sky,37.11,58.51,Mobile nazanin
8,salta,ar,59.00,clear sky,-24.79,-65.41,Hotel Alejandro I
11,rikitea,pf,77.45,broken clouds,-23.12,-134.97,Pension Maro'i
14,zhezkazgan,kz,57.92,scattered clouds,47.80,67.71,Kvartiry Posutochno.
16,portland,au,54.00,light rain,45.52,-122.68,Portland Marriott City Center
17,rikitea,pf,77.45,broken clouds,-23.12,-134.97,Pension Maro'i
18,kushima,jp,64.40,broken clouds,31.46,131.23,民泊 花おか
